In [168]:
import os
import glob
import json
from sqlalchemy import create_engine
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.petri.exporter import exporter as pnml_exporter
from pm4py.objects.petri.importer import importer as pnml_importer
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.petrinet import visualizer as pn_visualizer
from pm4py.evaluation.replay_fitness import evaluator as replay_fitness_evaluator


In [169]:
def obtem_conexao():
    DATABASES = {
        'dw_dev':{
            'NAME': 'dw_dev',
            'USER': 'minerador_processos',
            'PASSWORD': '5448minerador',
            'HOST': '10.3.192.85',
            'PORT': 5448,
        },
    }
    db = DATABASES['dw_dev']
    engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db['USER'],
    password = db['PASSWORD'],
    host = db['HOST'],
    port = db['PORT'],
    database = db['NAME'],
    )
    return create_engine(engine_string)

## Exporta Logs

In [170]:
def cria_sql(litigante,orgao) :
    sql_str = f"""select 
        ml.nr_processo, 
        ml.dt_lancamento_movimento, 
        ml.ds_movimento_complementos
    from 
        minerador_processos.mv_maiores_litigantes_arquivados ml
        join (
            -- k processos com menor duracao
            select 
                ml2.nr_processo, 
                ml2.dias_duracao_processo
            from 
                minerador_processos.mv_maiores_litigantes_arquivados ml2
            where 
    """
    if len(litigante) > 0:
        sql_str += f"""
           nr_doc_principal_pessoa ilike '{litigante}%' and
        """

    if len(orgao) > 0:
        sql_str += f"""
           cd_orgao_movimento = '{orgao}' and
        """
    sql_str += """
                -- Ajuste aqui a classe processual
                ml2.nr_classe_processual = 985
                --and ml2.nr_processo = '00105530220165030111'
            group by 
                ml2.nr_processo, 
                ml2.dias_duracao_processo
            order by ml2.dias_duracao_processo asc
            -- Ajuste aqui o numero de processos a sererm considerados
            -- limit 500
        ) melhores on melhores.nr_processo = ml.nr_processo
    group by ml.nr_processo, ml.dt_lancamento_movimento, ml.ds_movimento_complementos
    order by ml.nr_processo, ml.dt_lancamento_movimento
    """
    return sql_str

In [171]:
def obtem_varas_litigante(litigante):
    varas = []
    sql_str = f"""select 
        ml.cd_orgao
    from 
        minerador_processos.mv_maiores_litigantes_arquivados ml
        join (
            -- k processos com menor duracao
            select 
                ml2.nr_processo, 
                ml2.dias_duracao_processo
            from 
                minerador_processos.mv_maiores_litigantes_arquivados ml2
            where 
                -- Ajuste aqui o identificador do litigante
                nr_doc_principal_pessoa ilike '{litigante}%' and
                -- Ajuste aqui a classe processual
                ml2.nr_classe_processual = 985
                --and ml2.nr_processo = '00105530220165030111'
            group by 
                ml2.nr_processo, 
                ml2.dias_duracao_processo
            order by ml2.dias_duracao_processo asc
            -- Ajuste aqui o numero de processos a sererm considerados
            -- limit 500
        ) melhores on melhores.nr_processo = ml.nr_processo
    group by ml.cd_orgao
    """
    df_varas = pd.read_sql(sql_str,engine)
    varas = df_varas.values.tolist()
    varas = [vara for sublist in varas for vara in sublist]
    #print(varas)
    return varas

In [172]:
def exporta_log(df,caminho_exportacao_log):

    df_log = pd.DataFrame()
    df_log['case:concept:name'] = df['nr_processo'] 
    df_log['time:timestamp'] = df['dt_lancamento_movimento']
    df_log['concept:name'] = df['ds_movimento_complementos']
    #log_processos = dataframe_utils.convert_timestamp_columns_in_df(df_log)
    log_processos = df_log
    log_processos = log_processos.sort_values('time:timestamp')
    event_log_processos = log_converter.apply(log_processos)
    xes_exporter.apply(event_log_processos, caminho_exportacao_log)
    return event_log_processos

In [173]:
def exporta_log_varas(litigante,caminho_exportacao_log_litigante):
    varas_litigante = obtem_varas_litigante(litigante)
    for vara in varas_litigante:
        #print(vara)
        caminho_vara = caminho_exportacao_log_litigante+f'{vara}/'
        if not os.path.exists(caminho_vara):
            os.makedirs(caminho_vara)
        sql_str = cria_sql(litigante,str(vara))
        df = pd.read_sql(sql_str,engine)
        event_log_processos = exporta_log(df,caminho_vara + f'{litigante}V{vara}.xes')

In [174]:
def exporta_log_geral(caminho_exportacao):
    if not os.path.exists(caminho_exportacao):
        os.makedirs(caminho_exportacao)
    sql_str = cria_sql("","")
    df = pd.read_sql(sql_str,engine)
    event_log_processos = exporta_log(df,caminho_exportacao + 'ProcessoGeral.xes')

In [175]:
def exporta_logs(caminho_exportacao,engine,litigantes):
    print('Obtenção de logs')

    exporta_log_geral(caminho_exportacao)

    for dados_litigante in litigantes:
        litigante = dados_litigante[0]
        print(litigante)
        sql_str = cria_sql(litigante,'')
        df = pd.read_sql(sql_str,engine)
        caminho_exportacao_log = f'{caminho_exportacao}'+f'{litigante}/'
        if not os.path.exists(caminho_exportacao_log):
            os.makedirs(caminho_exportacao_log)
        event_log_processos = exporta_log(df,caminho_exportacao_log + f'{litigante}.xes')
        obtem_varas_litigante(litigante)
        exporta_log_varas(litigante,caminho_exportacao_log)

## Process Discovery

In [176]:
# inductive miner
def descobre_processo_im(event_log_processos,caminho_exportacao_processo,nome_arquivo):
    
    if nome_arquivo == '00360305':
        noise_thresold = 0.3
    else:
        noise_thresold = 0.2
        
    net, initial_marking, final_marking = inductive_miner.apply(event_log_processos, variant=inductive_miner.Variants.IMf,parameters={
            inductive_miner.Variants.IMf.value.Parameters.NOISE_THRESHOLD: noise_thresold})
    
    parameters = {pn_visualizer.Variants.WO_DECORATION.value.Parameters.FORMAT:"png"}
    gviz = pn_visualizer.apply(net, initial_marking, final_marking, parameters=parameters) 
    pn_visualizer.save(gviz, caminho_exportacao_processo + f'{nome_arquivo}.png')
    #pn_visualizer.view(gviz)
    gviz = pn_visualizer.apply(net, initial_marking, final_marking, parameters=parameters, variant=pn_visualizer.Variants.FREQUENCY, log=event_log_processos)
    pn_visualizer.save(gviz, caminho_exportacao_processo + f'{nome_arquivo}_frequencia.png')
    gviz = pn_visualizer.apply(net, initial_marking, final_marking, parameters=parameters, variant=pn_visualizer.Variants.PERFORMANCE, log=event_log_processos)
    pn_visualizer.save(gviz, caminho_exportacao_processo + f'{nome_arquivo}_desempenho.png')
    # exporta processos
    pnml_exporter.apply(net, initial_marking, caminho_exportacao_processo + f'{nome_arquivo}_petri_final.pnml', final_marking=final_marking)
#    pn_visualizer.view(gviz)

In [177]:
def descobre_processo_varas(litigante,caminho_log_litigante):
    varas_litigante = obtem_varas_litigante(litigante)
    for vara in varas_litigante:
        #print(vara)
        caminho_log_vara = caminho_log_litigante+f'{vara}/'
        caminho_arquivo_vara = caminho_log_vara+f'{litigante}V{vara}.xes'
        event_log_processos = xes_importer.apply(caminho_arquivo_vara)
        descobre_processo_im(event_log_processos,caminho_log_vara,f'{litigante}V{vara}')

In [178]:
def descobre_processo_geral(caminho_exportacao):
    caminho_arquivo_geral = caminho_exportacao+'ProcessoGeral.xes'
    event_log_processos = xes_importer.apply(caminho_arquivo_geral)
    descobre_processo_im(event_log_processos,caminho_exportacao,'ProcessoGeral')

In [179]:
def descobre_processos(caminho_exportacao):
    print('Descoberta de processos')
    descobre_processo_geral(caminho_exportacao)
    caminhos_log = glob.glob(f'{caminho_exportacao}*/')

    for caminho_log in caminhos_log:
        litigante = caminho_log.split('/')[-2]
        print(litigante)
        caminho_arquivo_log = f'{caminho_log}{litigante}.xes'
        event_log_processos = xes_importer.apply(caminho_arquivo_log)
        descobre_processo_im(event_log_processos,caminho_log,litigante)
        descobre_processo_varas(litigante,caminho_log)

# Fitness

In [180]:
def calcula_fitness(net, initial_marking, final_marking, caminho_arquivo_log, nome_arquivo_fitness):
    log = xes_importer.apply(caminho_arquivo_log)
    fitness = replay_fitness_evaluator.apply(log, net, initial_marking, final_marking, variant=replay_fitness_evaluator.Variants.ALIGNMENT_BASED)
    caminho_array = caminho_arquivo_log.split('/')
    del(caminho_array[-1])
    arquivo_fitness = "/".join(caminho_array)
    with open(f'{arquivo_fitness}/{nome_arquivo_fitness}.json', 'w') as outfile:
        json.dump(fitness, outfile)

In [181]:
def calcula_fitness_varas(litigante,caminho_log_litigante):
    print(f'Fitness das varas do {litigante}')
    net_litigante, initial_marking_litigante, final_marking_litigante = pnml_importer.apply(f'{caminho_log_litigante}{litigante}_petri_final.pnml')
    varas_litigante = obtem_varas_litigante(litigante)
    for vara in varas_litigante:
        #print(vara)
        caminho_log_vara = caminho_log_litigante+f'{vara}/'
        caminho_arquivo_vara = caminho_log_vara+f'{litigante}V{vara}.xes'
        nome_arquivo_fitness = f'FitnessLitigante{litigante}Vara{vara}'
        calcula_fitness(net_litigante, initial_marking_litigante, final_marking_litigante, caminho_arquivo_vara, nome_arquivo_fitness)

In [182]:
def calcula_fitness_processos(caminho_exportacao):
    print('Fitness de processos')

    # Processos litigantes x processo geral

    print('Fitness litigante x processo geral')
    net_geral, initial_marking_geral, final_marking_geral = pnml_importer.apply(f'{caminho_exportacao}ProcessoGeral_petri_final.pnml')

    caminhos_log = glob.glob(f'{caminho_exportacao}*/')
    for caminho_log in caminhos_log:
        litigante = caminho_log.split('/')[-2]
        print(litigante)
        caminho_arquivo_log = f'{caminho_log}{litigante}.xes'
        nome_arquivo_fitness = f'FitnessProcGeral{litigante}'
        #calcula_fitness(net_geral, initial_marking_geral, final_marking_geral, caminho_arquivo_log, nome_arquivo_fitness)
        calcula_fitness_varas(litigante,caminho_log)

In [183]:
def run():
    caminho_exportacao = '/home/dataeng/Inova/AnaliseLitigantes/'
    engine = obtem_conexao()
    litigantes = ['60701190','00360305']
    exporta_logs(caminho_exportacao,engine,dados_litigantes)
    descobre_processos(caminho_exportacao)
    calcula_fitness_processos(caminho_exportacao)


In [184]:
run()

Obtenção de logs
60701190
00360305
Descoberta de processos



60701190



00360305



Fitness de processos
Fitness litigante x processo geral
60701190
Fitness das varas do 60701190



00360305
Fitness das varas do 00360305


# Alinment

In [ ]:
# Alignment
#from pm4py.algo.discovery.inductive import algorithm as inductive_miner
#from pm4py.algo.conformance.alignments import algorithm as alignments

#net, initial_marking, final_marking = inductive_miner.apply(event_log_processos, variant=inductive_miner.Variants.IM)
#aligned_traces = alignments.apply_log(event_log_processos2, net, initial_marking, final_marking)
#print(aligned_traces)